# 1准备数据

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from pprint import pprint
from matplotlib import pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
from scipy.signal import savgol_filter #滤波
from sklearn.preprocessing import MinMaxScaler  
from torch.utils.data import Dataset,DataLoader
import torchkeras 

In [ ]:
from argparse import Namespace
from torch.optim import lr_scheduler
config = Namespace(
    file_path ="../alldata3.csv",
    batch_size = 128,
    dropout_p = 0.2,
    lr = 0.0001,
    optim_type = 'Adam',
    epochs = 200,
    ckpt_path = 'checkpoint',
    num_workers=0,
    name='transCNNtransformer',
    input_dim = 625,  # 转置卷积的输出维度
    embed_dim = 256,  # 嵌入维度
    num_heads = 8,  # 注意力头数
    num_layers = 2,  # Transformer 编码器的层数
    num_classes = 14,  # 分类类别数
    max_len = 5000  # 输入的最大序列长度
)
torch.manual_seed(17) #cpu
torch.cuda.manual_seed(17) #gpu
np.random.seed(17) #numpy

In [ ]:
class MyDataset(Dataset):
    def __init__(self,filepath):
        self.alldata=pd.read_csv(filepath,header=None)
        self.len=self.alldata.shape[0]
        self.alldata=np.array(self.alldata,dtype='float32')
        self.xdata=torch.from_numpy(self.alldata[:,0:-2])
        self.ydata=torch.from_numpy(self.alldata[:,[-1]])##多分类
    def __getitem__(self,index):
        xx=self.xdata[index]
        lb=savgol_filter(xx, window_length=7, polyorder=2)#Savitzky-Golay 平滑滤波器
        scaler=MinMaxScaler()
        lb=lb.reshape(-1,1)
        lb=scaler.fit_transform(lb)#层归一化
        lb=lb.reshape(1,-1)
        return lb,self.ydata[index]
    def __len__(self):
        return self.len

dfdata = MyDataset(config.file_path)

In [ ]:
#dataset
dfdata.ydata=dfdata.ydata.squeeze(1)#
dfdata.ydata=dfdata.ydata.to(dtype=torch.int64) #使用交叉熵做损失函数时
dftmp, dftest_raw = train_test_split(dfdata, random_state=40, test_size=0.1)
dftrain_raw, dfval_raw = train_test_split(dftmp, random_state=40, test_size=0.2)



In [ ]:
print("len(dftrain) = ",len(dftrain_raw))
print("len(dfval) = ",len(dfval_raw))
print("len(dftest) = ",len(dftest_raw))
print(dfdata.xdata.shape)
print(dfdata.ydata.shape)
print(type(dfdata.ydata[0].item()))

In [ ]:
#dataloader
dl_train =DataLoader(dftrain_raw, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
dl_val =DataLoader(dfval_raw, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)
dl_test =DataLoader(dftest_raw, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

In [ ]:
for features,labels in dl_val:
    break
print(features.shape)
print(labels.shape)
print(dl_train.__len__())

In [ ]:
# windows操作系统
plt.rcParams['font.sans-serif']=['SimHei']  # 用来正常显示中文标签 
plt.rcParams['axes.unicode_minus']=False  # 用来正常显示负号

# 创建模型

In [ ]:
import torch
import torch.nn as nn
import math
class DepthwiseSeparableConv1D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(DepthwiseSeparableConv1D, self).__init__()
        self.depthwise = nn.Conv1d(in_channels, in_channels, kernel_size=kernel_size, 
                                   groups=in_channels, padding=kernel_size//2)
        self.pointwise = nn.Conv1d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x
class TimeSeriesTransformer(nn.Module):
    def __init__(self, input_dim, embed_dim, num_heads, num_layers, num_classes, max_len=5000):
        super(TimeSeriesTransformer, self).__init__()
       #深度可分离卷积降维
        self.conv1=DepthwiseSeparableConv1D(in_channels=1,out_channels=32,kernel_size = 11)
        self.relu=nn.ReLU()
        self.maxpool= nn.MaxPool1d(2)#等价于nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2=DepthwiseSeparableConv1D(in_channels=32,out_channels=64,kernel_size = 21)
        self.conv3=DepthwiseSeparableConv1D(in_channels=64,out_channels=128,kernel_size = 41)
         # 线性投影层，将输入数据5000映射到 embed_dim 维度
        self.embedding = nn.Linear(input_dim, embed_dim)
        
        # 位置编码
        self.pos_encoder = PositionalEncoding(embed_dim, max_len)
        
        # Transformer 编码器层
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads,
                                                   dim_feedforward=256, dropout=0.2)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        
        # 全局池化（或者可以选用 [CLS] token 表示）
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)
        
        # 最终分类层
        self.fc = nn.Linear(embed_dim, num_classes)
    
    def forward(self, x):
#         x = x.permute(0, 2, 1)
        #转置卷积forward
        x=self.conv1(x)
        x=self.relu(x)
        x=self.maxpool(x)
        
        x=self.conv2(x)
        x=self.relu(x)
        x=self.maxpool(x)
        
        x=self.conv3(x)
        x=self.relu(x)
        x=self.maxpool(x)
        # print(x.shape)
        # 输入 x 的形状：[batch_size, seq_len, input_dim]
        x = self.embedding(x)  # 形状变为：[batch_size, seq_len, embed_dim]
        x = self.pos_encoder(x)  # 添加位置编码
        x = x.permute(1, 0, 2)  # Transformer 期望的输入形状：[seq_len, batch_size, embed_dim]
        
        # 通过 Transformer 编码器
        x = self.transformer_encoder(x)  # 输出形状：[seq_len, batch_size, embed_dim]
        x = x.permute(1, 2, 0)  # 调整形状为：[batch_size, embed_dim, seq_len]
        
        # 全局平均池化
        x = self.global_avg_pool(x).squeeze(-1)  # 形状变为：[batch_size, embed_dim]
        
        # 全连接分类层
        x = self.fc(x)  # 输出形状：[batch_size, num_classes]
        return x

class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # [1, max_len, embed_dim]
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]  # 加上位置编码

In [ ]:
class multiAccuracy(nn.Module):
    """Accuracy for multi-classification task."""
    def __init__(self):
        """Initialize the Accuracy module."""
        super().__init__()
        # Counters for correct and total predictions
        self.correct = nn.Parameter(torch.tensor(0.0), requires_grad=False)
        self.total = nn.Parameter(torch.tensor(0.0), requires_grad=False)

    def forward(self, preds: torch.Tensor, targets: torch.Tensor):
        preds = preds.argmax(dim=-1)
        targets = targets.reshape(-1)
        m = (preds == targets).sum()
        n = targets.shape[0] 
        self.correct += m 
        self.total += n
        
        return m/n

    def compute(self):
         return self.correct.float() / self.total 

    def reset(self):
        self.correct -= self.correct
        self.total -= self.total

In [ ]:
from torchkeras import summary
net = TimeSeriesTransformer(config.input_dim, config.embed_dim, config.num_heads, 
                            config.num_layers, config.num_classes, config.max_len)
summary(net,input_data=features);

# 验证集参数计算

In [ ]:
model.evaluate(dl_test)

support：当前行的类别在测试数据中的样本总量，如上表就是，在class 0 类别在测试集中总数量为1；<br>
precision：精度=正确预测的个数(TP)/被预测正确的个数(TP+FP)；人话也就是模型预测的结果中有多少是预测正确的<br>
ecall:召回率=正确预测的个数(TP)/预测个数(TP+FN)；人话也就是某个类别测试集中的总量，有多少样本预测正确了；<br>
f1-score:F1 = 2*精度*召回率/(精度+召回率)<br>
micro avg：计算所有数据下的指标值，假设全部数据 5 个样本中有 3 个预测正确，所以 micro avg 为 3/5=0.6<br>
macro avg：每个类别评估指标未加权的平均值，比如准确率的 macro avg，(0.50+0.00+1.00)/3=0.5<br>
weighted avg：加权平均，就是测试集中样本量大的，给他设置的权重大点；比如第一个值的计算方法，(0.50*1 + 0.0*1 + 1.0*3)/5 = 0.70。更好点

In [ ]:
#加载网络
net2 = TimeSeriesTransformer(config.input_dim, config.embed_dim, config.num_heads, 
                            config.num_layers, config.num_classes, config.max_len) 
net2.load_state_dict(torch.load('checkpoint_multi-transformer'))

# 评估矩阵

In [ ]:
from tqdm import tqdm 
from sklearn.metrics import classification_report
net2 = net2.cpu()
net2.eval()
preds = []
believer=[]
ytrue=[]
with torch.no_grad():
    for batch in tqdm(dl_test):
        inputs,labels=batch
#         print(inputs.shape)
#         print(labels.shape)
        
        x=net2(inputs)
#         print(x.shape)
#         be= F.softmax(x, dim=1)
        believer.extend(x.tolist())
        preds.extend(x.argmax(dim=-1).tolist())
        ytrue.extend(labels.tolist())
    pprint(len(preds)) 
    pprint(len(ytrue))
    pprint(len(believer))
#     pprint(preds) 
#     pprint(yhat)
reportt=classification_report(ytrue,preds)
print(reportt)#打印每个类别的指标

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

# 计算混淆矩阵
cm = confusion_matrix(ytrue,preds)
# 将混淆矩阵转换为DataFrame
df_cm = pd.DataFrame(cm, index=['True {}'.format(i) for i in range(cm.shape[0])],columns=['Predicted {}'.format(i) for i in range(cm.shape[1])])
# pprint(df_cm)
# pprint(cm)


In [ ]:
plt.figure(figsize=(20, 20))
ax=plt.matshow(cm , cmap=plt.cm.Blues,fignum=1, aspect='auto') 
cbar=plt.colorbar(ax, fraction=0.05, pad=0.04)
cbar.ax.tick_params(labelsize=20)
for i in range(len(cm )): 
    for j in range(len(cm )):
        plt.annotate(cm [i,j], xy=(i, j), horizontalalignment='center', verticalalignment='center',fontsize=20)
plt.ylabel('真实值标签',fontsize=40)
plt.xlabel('预测值标签',fontsize=40) 
plt.tick_params(axis='both', labelsize=30)
plt.savefig('multi-混淆.jpg'.format(), dpi=120, bbox_inches='tight')
plt.show()

In [ ]:
#PR
import math
result_matrix = [[math.exp(element) for element in row] for row in believer] 


In [ ]:
# 1二分类标注
# df = pd.DataFrame()
tempp=np.array(ytrue)
y_test =(tempp==0)
#2二分类预测置信度
tempr=np.array(result_matrix)
y_score = tempr[:,0]
y_score
#3计算ap
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
precision, recall, thresholds = precision_recall_curve(y_test, y_score)
AP = average_precision_score(y_test, y_score, average='weighted')
AP

In [ ]:
#4计算auc
from sklearn.metrics import roc_curve, auc
fpr, tpr, threshold = roc_curve(y_test, y_score)

In [ ]:
from matplotlib import colors as mcolors
import random
random.seed(124)
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan', 'black', 'indianred', 'brown', 'firebrick', 'maroon', 'darkred', 'red', 'sienna', 'chocolate', 'yellow', 'olivedrab', 'yellowgreen', 'darkolivegreen', 'forestgreen', 'limegreen', 'darkgreen', 'green', 'lime', 'seagreen', 'mediumseagreen', 'darkslategray', 'darkslategrey', 'teal', 'darkcyan', 'dodgerblue', 'navy', 'darkblue', 'mediumblue', 'blue', 'slateblue', 'darkslateblue', 'mediumslateblue', 'mediumpurple', 'rebeccapurple', 'blueviolet', 'indigo', 'darkorchid', 'darkviolet', 'mediumorchid', 'purple', 'darkmagenta', 'fuchsia', 'magenta', 'orchid', 'mediumvioletred', 'deeppink', 'hotpink']
markers = [".",",","o","v","^","<",">","1","2","3","4","8","s","p","P","*","h","H","+","x","X","D","d","|","_",0,1,2,3,4,5,6,7,8,9,10,11]
linestyle = ['--', '-.', '-']
def get_line_arg():
    '''
    随机产生一种绘图线型
    '''
    line_arg = {}
    line_arg['color'] = random.choice(colors)
    # line_arg['marker'] = random.choice(markers)
    line_arg['linestyle'] = random.choice(linestyle)
    line_arg['linewidth'] = random.randint(1, 4)
    # line_arg['markersize'] = random.randint(3, 5)
    return line_arg

In [ ]:
get_line_arg()

In [ ]:
classes=[0,1,2,3,4,5,6,7,8,9,10,11,12,13]


In [ ]:
plt.figure(figsize=(15, 10))
plt.xlim([-0.01, 1.0])
plt.ylim([0.0, 1.01])
# plt.plot([0, 1], [0, 1],ls="--", c='.3', linewidth=3, label='随机模型')
plt.xlabel('召回率',fontsize=25)
plt.ylabel('精确率',fontsize=25)
plt.rcParams['font.size'] = 22
# plt.grid(True)

ap_list = []
for each_class in classes:
    y_test = list((tempp == each_class))
#     print(len(y_test),each_class)
    y_score = list(tempr[:,each_class])
    precision, recall, thresholds = precision_recall_curve(y_test, y_score)
    AP = average_precision_score(y_test, y_score, average='weighted')
    plt.plot(recall, precision, **get_line_arg(), label='label:'+str(each_class))
    plt.legend()
    ap_list.append(AP)

plt.legend(loc='best', fontsize=15)
plt.savefig('multi-各类别PR曲线2.jpg'.format(), dpi=120, bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=(14, 10))
plt.xlim([-0.01, 1.0])
plt.ylim([0.0, 1.01])
# plt.plot([0, 1], [0, 1],ls="--", c='.3', linewidth=3, label='随机模型')
plt.xlabel('假正率')
plt.ylabel('真正率')
plt.rcParams['font.size'] = 22
# plt.grid(True)

auc_list = []
for each_class in classes:
    y_test = list((tempp == each_class))
    y_score = list(tempr[:,each_class])
    fpr, tpr, threshold = roc_curve(y_test, y_score)
    plt.plot(fpr, tpr, **get_line_arg(), label='label:'+str(each_class))
    plt.legend()
    auc_list.append(auc(fpr, tpr))

plt.legend(loc='lower right', fontsize=15)
plt.savefig('multi-各类别ROC曲线.jpg'.format(), dpi=120, bbox_inches='tight')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
auc(fpr, tpr)

In [ ]:
#保存1
report=classification_report(ytrue,preds,target_names=classes, output_dict=True)
del report['accuracy']
df_report = pd.DataFrame(report).transpose()

df_report

In [ ]:
# 计算 ap值 的 宏平均 和 加权平均
macro_avg_auc = np.mean(ap_list)
# print(len(ap_list))
weighted_avg_auc = sum(ap_list * df_report.iloc[:-2]['support'] / len(ytrue))
ap_list.append(macro_avg_auc)
ap_list.append(weighted_avg_auc)
# print(len(ap_list))
df_report['AP'] = ap_list
df_report

In [ ]:
# 计算 AUC值 的 宏平均 和 加权平均
macro_avg_auc = np.mean(auc_list)
weighted_avg_auc = sum(auc_list * df_report.iloc[:-2]['support'] / len(ytrue))
auc_list.append(macro_avg_auc)
auc_list.append(weighted_avg_auc)
df_report['AUC'] = auc_list
df_report

In [ ]:
df_report.to_csv('各类别准确率评估指标.csv', index_label='类别')
# df_report.to_csv('各类别准确率评估指标.csv', index=False)

# 混淆矩阵

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

# 计算混淆矩阵
cm = confusion_matrix(ytrue,preds)
# 将混淆矩阵转换为DataFrame
df_cm = pd.DataFrame(cm, index=['True {}'.format(i) for i in range(cm.shape[0])],columns=['Predicted {}'.format(i) for i in range(cm.shape[1])])
pprint(df_cm)
pprint(cm)


In [ ]:
from sklearn.metrics import cohen_kappa_score
kappa = cohen_kappa_score(ytrue,preds)
kappa

In [ ]:
# 1!!!!!!!!!!!!!使用seaborn绘制混淆矩阵、seaborn版本过老
plt.figure(figsize=(10,7))
sns.heatmap(df_cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.show()

In [ ]:
#2# 使用matplotlib
plt.figure(figsize=(20, 20))
ax=plt.matshow(cm , cmap=plt.cm.Blues,fignum=1, aspect='auto') 
cbar=plt.colorbar(ax, fraction=0.05, pad=0.04)
cbar.ax.tick_params(labelsize=16)
for i in range(len(cm )): 
    for j in range(len(cm )):
        plt.annotate(cm [i,j], xy=(i, j), horizontalalignment='center', verticalalignment='center',fontsize=20)
plt.ylabel('True label',fontsize=40)
plt.xlabel('Predicted label',fontsize=40) 
plt.show()